# 分层索引

- `MultiIndex`
- `stack()`
- `unstack()`
- `swaplevel()`
- `sort_index()`
- `set_index()`
- `reset_index()`

分层索引实现了多层级的坐标功能。

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.318444
   2    0.227006
   3   -1.027677
b  1   -0.663820
   3   -0.599214
c  1    0.307556
   2    0.607584
d  2   -0.891579
   3    0.758960
dtype: float64

分层索引的索引类型为`MultiIndex`。

In [4]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

使用分层索引可以对`Series`的一个或多个元素进行索引。

In [5]:
data['b']

1   -0.663820
3   -0.599214
dtype: float64

In [7]:
data['b': 'd']

b  1   -0.663820
   3   -0.599214
c  1    0.307556
   2    0.607584
d  2   -0.891579
   3    0.758960
dtype: float64

In [8]:
data[['b', 'c']]

b  1   -0.663820
   3   -0.599214
c  1    0.307556
   2    0.607584
dtype: float64

分层索引还支持根据次级索引进行索引操作，如检索所有次级索引为2的行。

In [9]:
data.loc[:, 2]

a    0.227006
c    0.607584
d   -0.891579
dtype: float64

`unstack()`方法能够将一个具有分层索引的`Series`重新排列为新的`DataFrame`。

In [10]:
data.unstack()

,1,2,3
a,-0.318444,0.227006,-1.027677
b,-0.663820,NaN,-0.599214
c,0.307556,0.607584,NaN
d,NaN,-0.891579,0.758960


`stack()`方法为`unstack()`方法的逆操作。

In [11]:
data.unstack().stack()

a  1   -0.318444
   2    0.227006
   3   -1.027677
b  1   -0.663820
   3   -0.599214
c  1    0.307556
   2    0.607584
d  2   -0.891579
   3    0.758960
dtype: float64

`DataFrame`中同样支持分层索引。

In [12]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'],
                             ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

`MultiInedx`类型可以通过`names`属性为每一级索引命名。

In [17]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

也可以直接创建`MultiIndex`对象。

In [20]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], 
                        ['Green', 'Red', 'Green']],
                       names=['state', 'color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

## 分层索引重排

分层索引的某一级需要重新排列。

`swaplevel()`方法可以将指定的索引层级进行互换。

In [21]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

`sort_index()`方法能够根据`index`的某一级索引的值对数据进行排序。

In [25]:
# 根据key1列的值对数据进行排序
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

## 分层求和数据

通过指定聚合函数的`level`参数值，能够对数据行或者列的不同层级进行聚合操作。

In [26]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [27]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

该功能实际上是通过`groupby()`方法实现的。

## `DataFrame`的列索引

`set_index()`方法通过`DataFrame`的一个或多个列的值作为对应行的`index`创建一个新的`DataFrame`。

In [28]:
frame = pd.DataFrame({'a': range(7), 
                      'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [29]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [31]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

`reset_index()`方法是`set_index()`方法的逆操作，将分层索引作为新的列添加到`DataFrame`中。

In [32]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
